# 200968130

### The Game  
According to the “Six Degrees of Kevin Bacon" game, anyone in the Hollywood film industry  can  be  connected to  Kevin Bacon within  six  steps, where  each step  consists of finding a film that two actors both starred in.To solve the problem, find the shortest path between  any  two  actors  by  choosing  a  sequence  of  movies  that  connects  them.  
For example, the shortest path between Jennifer Lawrence and Tom Hanks is 2: Jennifer Lawrence is connected to Kevin Bacon by both starring in “X-Men:  First Class,” and Kevin Bacon is connected to Tom Hanks by both starring in “Apollo 13”. 

###Problem Solving Agent:
Given two  actors nodes in  the  graph  we  need  to  find the  distance  (shortest  path) between the nodes. Write a python program to determinehow many “degrees of separation” apart two actors are. Find thedistance or the degree of separation., using a.Breadth first searchb.Depth first search

### Distribution Code: 
Data & Download the distribution code from:https://cdn.cs50.net/ai/2020/x/projects/0/degrees.zip 

The distribution code contains two sets of CSV data files: one set in the large directory and one   set in   the   small   directory. Use   the smalldataset   for   ease   of   testing   and experimentation.Each dataset consists of three CSV files.
- 1.small/people.csv:each  person  has  a  unique  id,  corresponding  with  their  id  in IMDb’s database. They also have a name, and a birth year.
- 2.small/movies.csv:You’ll see here that each movie also has a unique id, in addition to a title and the year in which the movie was released.
- 3.small/stars.csv:This file establishes a relationship between the people   in people.csv and the movies in movies.csv. Each row is a pair of a person_id value and movie_id value. 

For example: The first row (ignoring the header), for example, states that the person with id 102 starred in the movie with id 104257. Checking that against people.csv and movies.csv, you’ll find that this line is saying that Kevin Bacon starred in the movie “A Few Good Men.”

In [7]:
!unzip degrees.zip

Archive:  degrees.zip
   creating: degrees/
   creating: degrees/large/
  inflating: degrees/large/stars.csv  
  inflating: degrees/large/people.csv  
  inflating: degrees/large/movies.csv  
  inflating: degrees/util.py         
  inflating: degrees/degrees.py      
   creating: degrees/small/
  inflating: degrees/small/stars.csv  
  inflating: degrees/small/people.csv  
  inflating: degrees/small/movies.csv  


In [9]:
import pandas as pd
movies = pd.read_csv('/content/degrees/small/movies.csv')
people = pd.read_csv('/content/degrees/small/people.csv')
stars = pd.read_csv('/content/degrees/small/stars.csv')

In [10]:
movies

,id,title,year
0,112384,Apollo 13,1995
1,104257,A Few Good Men,1992
2,109830,Forrest Gump,1994
3,93779,The Princess Bride,1987
4,95953,Rain Man,1988


In [11]:
people

,id,name,birth
0,102,Kevin Bacon,1958
1,129,Tom Cruise,1962
2,144,Cary Elwes,1962
3,158,Tom Hanks,1956
4,1597,Mandy Patinkin,1952
5,163,Dustin Hoffman,1937
6,1697,Chris Sarandon,1942
7,193,Demi Moore,1962
8,197,Jack Nicholson,1937
9,200,Bill Paxton,1955


In [12]:
stars

,person_id,movie_id
0,102,104257
1,102,112384
2,129,104257
3,129,95953
4,144,93779
5,158,109830
6,158,112384
7,1597,93779
8,163,95953
9,1697,93779


### degrees.py:
At the top, several data structures are defined to store information from the CSV files. The names dictionary is a way to look up a person by their name: it maps names to a set of corresponding ids (because it’s possible that multiple actors have the same name). The people dictionary maps each person’s id to another dictionary with values for the person’s name, birth year, and the set of all the movies they have starred in. 

And the movies dictionary maps each movie’s id to another dictionary with values for that movie’s title, release year, and the set of all the movie’s stars. The load_data function loads data from the CSV files into these data structures.The main function in this program first loads data into memory (the directory from which  the  data  is  loaded  can  be  specified  by  a  command-line  argument).  Then,  the function  prompts the user  to type  in  two  names.  The  person_id_for_name  function retrieves the id for any person (and handles prompting the user to clarify, in the event that multiple people have the same name). The function then calls the shortest_path function  to  compute  the  shortest  path  between  the  two  people,  and  prints  out  the path.

In [13]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action


class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node


class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

In [18]:
import csv
import sys
# from util import Node, StackFrontier, QueueFrontier

# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}


def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


def main():
    # if len(sys.argv) > 2:
    #     sys.exit("Usage: python degrees.py [directory]")
    directory = '/content/degrees/small'

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
      print('Notfound')
      return
    target = person_id_for_name(input("Name: "))
    if target is None:
      print('Notfound')
      return
    method = input('Method Name: ')
    path = shortest_path(source, target,method)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")


def shortest_path(source, target,method):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.
    If no possible path, returns None.
    """
    if method=='bfs':
      start = Node(source, None, None)
      frontier = QueueFrontier()
      frontier.add(start)

      # Initialise an empty explored set to hold explored states (actors):
      explored = set()

      # Loop until a solution is found, or Frontier is empty(no solution):
      while True:

        if len(explored) % 100 == 0:
          print('Actors explored to find solution: ', len(explored))
          print('Nodes left to expand in Frontier: ', len(frontier.frontier))

        # Check for empty Frontier and return with no path if empty
        if frontier.empty():
          print('Frontier is Empty - No Connection Between Actors!')
          print(len(explored), 'actors explored to with no solution found!')
          return None

        # Otherwise expand the next node in the Queue, add it to the explored states and get set of movies and actors for the actor in the current node:
        curr_node = frontier.remove()
        explored.add(curr_node.state)

        for action, state in neighbors_for_person(curr_node.state):

          # If state (actor) is the target actor then solution has been found, return path:
          if state == target:
            print('Solution Found!')
            print(len(explored), 'actors explored to find solution!')
            # Create path from source to target
            path = []
            path.append((action, state))

            # Add action and state to path until back to start node
            while curr_node.parent != None:
              path.append((curr_node.action, curr_node.state))
              curr_node = curr_node.parent

            path.reverse()

            return path

          # Otherwise add the new states to explore to the frontier:
          if not frontier.contains_state(state) and state not in explored:
            new_node = Node(state, curr_node, action)
            frontier.add(new_node)
    if method=='dfs':
      start = Node(source, None, None)
      frontier = StackFrontier()
      frontier.add(start)

      # Initialise an empty explored set to hold explored states (actors):
      explored = set()

      # Loop until a solution is found, or Frontier is empty(no solution):
      while True:

        if len(explored) % 100 == 0:
          print('Actors explored to find solution: ', len(explored))
          print('Nodes left to expand in Frontier: ', len(frontier.frontier))

        # Check for empty Frontier and return with no path if empty
        if frontier.empty():
          print('Frontier is Empty - No Connection Between Actors!')
          print(len(explored), 'actors explored to with no solution found!')
          return None

        # Otherwise expand the next node in the Queue, add it to the explored states and get set of movies and actors for the actor in the current node:
        curr_node = frontier.remove()
        explored.add(curr_node.state)

        for action, state in neighbors_for_person(curr_node.state):

          # If state (actor) is the target actor then solution has been found, return path:
          if state == target:
            print('Solution Found!')
            print(len(explored), 'actors explored to find solution!')
            # Create path from source to target
            path = []
            path.append((action, state))

            # Add action and state to path until back to start node
            while curr_node.parent != None:
              path.append((curr_node.action, curr_node.state))
              curr_node = curr_node.parent

            path.reverse()

            return path

          # Otherwise add the new states to explore to the frontier:
          if not frontier.contains_state(state) and state not in explored:
            new_node = Node(state, curr_node, action)
            frontier.add(new_node)
    raise NotImplementedError


def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors



Added option to add method name used for search:
 - bfs
 - dfs

In [17]:
main()

Loading data...
Data loaded.
Name: Kevin Bacon
Name: Tom Hanks
Method Name: dfs
Actors explored to find solution:  0
Nodes left to expand in Frontier:  1
Solution Found!
1 actors explored to find solution!
1 degrees of separation.
1: Kevin Bacon and Tom Hanks starred in Apollo 13
